# Project 3: Airbnb
**This is the third of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-12-05, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python/PySpark**
- **For each question you may use as many cells for your solution as you like**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**

# Introduction
[Airbnb](http://airbnb.com) is an online marketplace for arranging or offering lodgings. In this project you will use Spark to analyze data obtained from the Airbnb website. The purpose of the analysis is to extract information about trends and patterns from the data.

The project has two parts.

### Part 1: Loading, describing and preparing the data
There's quite a lot of data. Make sure that you can load and correctly parse the data, and that you understand what the dataset contains. You should also prepare the data for the analysis in part two. This means cleaning it and staging it so that subsequent queries are fast.

### Par 2: Analysis
In this part your goal is to learn about trends and usage patterns from the data. You should give solutions to the tasks defined in this notebook, and you should use Spark to do the data processing. You may use other libraries like for instance Pandas and matplotlib for visualisation.

## Guidelines
- Processing data should be done using Spark. Once data has been reduced to aggregate form, you may use collect to extract it into Python for visualisation.
- Your solutions will be evaluated by correctness, code quality and interpretability of the output. This means that you have to write clean and efficient Spark code that will generate sensible execution plans, and that the tables and visualisations that you produce are meaningful and easy to read.
- You may add more cells for your solutions, but you should not modify the notebook otherwise.

### Create Spark session and define imports

In [1]:
from pyspark.sql import *
from pyspark.sql import functions as f
from pyspark.sql.types import *

spark = SparkSession.builder.appName("SparkIntro").getOrCreate()

# Part 1: Loading, describing and preparing the data
The data comes in two files. Start by downloading the files and putting them in your `data/` folder.

- [Listings](https://files.dtu.dk/u/siPzAasj8w2gI_ME/listings.csv?l) (5 GB)
- [Reviews](https://files.dtu.dk/u/k3oaPYp6GjKBeho4/reviews.csv?l) (9.5 GB)

### Load the data
The data has multiline rows (rows that span multiple lines in the file). To correctly parse these you should use the `multiline` option and set the `escape` character to be `"`.

In [3]:
import itertools
import csv
import pandas as pd
import pyspark.sql.functions as f
from functools import reduce
from pyspark.sql.types import *
from pyspark.sql.functions import substring, length, col, expr

##### Slicing the data

In [ ]:


entries = []
with open('/home/jovyan/work/Project3/listings.csv', 'r') as f:
    mycsv = csv.reader(f)
    for row in itertools.islice(mycsv, 10000):
        entries.append(row)

columns_ = entries[0]
df = pd.DataFrame(entries,columns=columns_)
df = df.iloc[1:-1,:]
df.head()
#df.to_csv("listings_sub_10000.csv")

##### Reading into spark

In [ ]:
df = spark.read.option("delimiter",",").option("multiline",True).option("header", True).option("escape",'"').option("inferSchema",True).csv('listings_sub_10000.csv')


In [ ]:
df.show()

In [ ]:
df.select("listing_url").show()

### Describe the data
List the features (schema) and sizes of the datasets.

In [ ]:
df.printSchema()

In [ ]:
print("rows: ", df.count(), "columns: ", len(df.columns))

In [ ]:
#Removing all rows which has null
df.where(reduce(lambda x, y: x & y,  (f.col(x).isNotNull() for x in df.columns))).count()

In [ ]:
#Removing all rows which has null
df.dropna(how='any').count()

### Prepare the data for analysis
You should prepare two dataframes to be used in the analysis part of the project. You should not be concerned with cleaning the data. There's a lot of it, so it will be sufficient to drop rows that have bad values. You may want to go back and refine this step at a later point when doing the analysis.

You may also want to consider if you can stage your data so that subsequent processing is more efficient (this is not strictly necessary for Spark to run, but you may be able to decrease the time you sit around waiting for Spark to finish things)

# Part 2: Analysis
Use Spark and your favorite tool for data visualization to solve the following tasks.

## The basics
Compute and show a dataframe with the number of listings and neighbourhoods per city.

Based on the table above, you should choose a city that you want to continue your analysis for. The city should have mulitple neighbourhoods with listings in them.

Compute and visualize the number of listings of different property types per neighbourhood in your city.

## Prices
Compute the minimum, maximum and average listing price in your city. 

In [ ]:
city = "Stockholm"

In [ ]:
df_s = df.filter(f.col("city")==city)

In [ ]:
df_s.count()

In [ ]:
#df_s.price = df_s.price.cast('float')

In [ ]:
from pyspark.sql.types import *

In [ ]:
from pyspark.sql.functions import substring, length, col, expr
df_s = df_s.withColumn("price",expr("substring(price, 2, length(price))"))

In [ ]:
from  pyspark.sql.functions import regexp_replace,col

In [ ]:
df_s = df_s.withColumn("price", regexp_replace(f.col("price"), ",", ""))

In [ ]:
#df_s.show()

In [ ]:
#df_s.select(df_s.price.cast("float").alias('price_new')).collect()

In [ ]:
from pyspark.sql.types import *
df_d = df_s.withColumn("price",df_s["price"].cast(DoubleType()))

In [ ]:
#df_d.show()

In [ ]:
print("Basic statistics for", city)
df_d.select(f.avg('price').alias("Averge price"), f.max('price').alias("Max price"), f.min('price').alias("Min price")).show()

Compute and visualize the distribution of listing prices in your city.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
sns.set(rc={'figure.figsize':(16,6)})

In [ ]:
df_city_p = df_d.toPandas()

In [ ]:
sns.distplot(df_city_p.price, kde=False, rug=False, bins=150);

The value of a listing is its rating divided by its price.

Compute and show a dataframe with the 3 highest valued listings in each neighbourhood.

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
df_d = df_d.withColumn("review_scores_rating",df_s["review_scores_rating"].cast(DoubleType()))

In [ ]:
df_d = df_d.withColumn("value_listing",lit(f.col("review_scores_rating")/f.col("price")))

In [ ]:
df_value = df_d.select(f.col("city"),f.col("neighbourhood"),(f.col("value_listing")))

In [ ]:
#df_value.orderBy(df_value.value_listing.desc()).show()

In [ ]:
sorted_by_value = Window.partitionBy('neighbourhood').orderBy(f.desc('value_listing'))

In [ ]:
ranked_by_value = df_d.withColumn('value_listing_rank', f.rank().over(sorted_by_value))

In [ ]:
ranked_df = ranked_by_value.filter(f.col('value_listing_rank') <= 3).drop('value_listing_rank').orderBy('neighbourhood', f.desc('value_listing'))

In [ ]:
ranked_df.select(f.col("neighbourhood"),f.col("value_listing"),f.col("city"),f.col("name")).show(50)

## Trends
Now we want to analyze the "popularity" of your city. The data does not contain the number of bookings per listing, but we have a large number of reviews, and we will assume that this is a good indicator of activity on listings.

Compute and visualize the popularity (i.e., number of reviews) of your city over time.

Compute and visualize the popularity of neighbourhoods over time. If there are many neighbourhoods in your city, you should select a few interesting ones for comparison.

Compute and visualize the popularity of your city by season. For example, visualize the popularity of your city per month.

## Reviews
In this part you should determine which words used in reviews that are the most positive. 

The individual reviews do not have a rating of the listing, so we will assume that each review gave the average rating to the listing, i.e., the one on the listing.

You should assign a positivity weight to each word seen in reviews and list the words with the highest weight. It is up to you to decide what the weight should be. For example, it can be a function of the rating on the listing on which it occurs, the number of reviews it occurs in, and the number of unique listings for which it was used to review.

Depending on your choice of weight function, you may also want to do some filtering of words. For example, remove words that only occur in a few reviews.

In [ ]:
import itertools
import csv

entries = []
with open('../Project3/reviews.csv', 'r') as f:
    mycsv = csv.reader(f)
    for row in itertools.islice(mycsv, 10000):
        entries.append(row)
        
import pandas as pd
columns_ = entries[0]
df = pd.DataFrame(entries,columns=columns_)
df = df.iloc[1:-1,:]
#df.head()
#df.to_csv("reviews_sub_10000.csv")

In [3]:
import pandas as pd

In [2]:
r = pd.read_csv("reviews_sub_10000.csv")
l = pd.read_csv("listings_sub_10000.csv")

NameError: name 'pd' is not defined

In [ ]:
r.head()

In [ ]:
l.head()

From the listings we need to:
- calculate the average rating for each ID
- join the average rating on to the ratings

In [89]:
dfl = spark.read.option("delimiter",",").option("multiline",True).option("header", True).option("escape",'"').option("inferSchema",True).csv('listings_sub_10000.csv')

In [90]:
dfr = spark.read.option("delimiter",",").option("multiline",True).option("header", True).option("escape",'"').option("inferSchema",True).csv('reviews_sub_10000.csv')

In [91]:
dfr.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- listing_id: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- reviewer_id: integer (nullable = true)
 |-- reviewer_name: string (nullable = true)
 |-- comments: string (nullable = true)



In [92]:
from pyspark.sql.functions import col, lower, regexp_replace, split

def clean_text(c):
    c = lower(c)
    c = regexp_replace(c, "(https?\://)\S+", "")
    c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
    return c

In [93]:
dfr = dfr.withColumn("text_clean", clean_text(col("comments")))

In [94]:
dfr.select(f.col("text_clean")).show()

+--------------------+
|          text_clean|
+--------------------+
|prima plek om sto...|
|cosy and clean fl...|
|the host canceled...|
|kims place was ou...|
|great spacious ap...|
|kim is a very fri...|
|the apartment is ...|
|nicely appointed ...|
|it was a pleasure...|
|kims place is sim...|
|we absolutely lov...|
|amazing place to ...|
|very pleasant sta...|
|we had a wonderfu...|
|great location co...|
|lovely flat in a ...|
|kims flat is in a...|
|i would definitel...|
|kims place is per...|
|the flat is so co...|
+--------------------+
only showing top 20 rows



In [95]:
#mean_scores = dfl.groupBy("id").agg(f.mean("review_scores_rating").alias("average_rating"))
#mean_scores = mean_scores.select(f.col("id").alias("id_"), f.col("average_rating"))

In [96]:
dfl = dfl.select(f.col("id").alias("id_"), f.col("review_scores_rating").alias("average_rating"))

In [97]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [98]:
dfl.show()

+------+--------------+
|   id_|average_rating|
+------+--------------+
|145320|            97|
|155220|          null|
|155685|            94|
|164448|            97|
|170651|            93|
|206221|            98|
|220233|          null|
|220851|            93|
|242188|            97|
|259025|            89|
|259722|            92|
|273906|           100|
|274876|            99|
|278311|            92|
|283446|            97|
|299154|            92|
|301680|            92|
|302496|            99|
|313195|            96|
|313763|            98|
+------+--------------+
only showing top 20 rows



In [99]:
dfr = dfr.join(dfl, (f.col('listing_id') == f.col('id_')), 'inner').drop('id_')

In [100]:
dfr.show()

+----+----------+--------+-------------------+-----------+------------------+--------------------+--------------------+--------------+
| _c0|listing_id|      id|               date|reviewer_id|     reviewer_name|            comments|          text_clean|average_rating|
+----+----------+--------+-------------------+-----------+------------------+--------------------+--------------------+--------------+
|9766|   1271843| 8513212|2013-11-03 00:00:00|    4641577|            Pamela|André was my firs...|andr was my first...|            99|
|9767|   1271843| 9043505|2013-12-02 00:00:00|    6890450|              Emma|Andre was the bes...|andre was the bes...|            99|
|9768|   1271843| 9533868|2014-01-01 00:00:00|    9496791|   Julie And Chris|André was a fanta...|andr was a fantas...|            99|
|9769|   1271843|10049736|2014-01-27 00:00:00|     940929|             Denis|It was a right ch...|it was a right ch...|            99|
|9770|   1271843|10324840|2014-02-13 00:00:00|   114540

In [101]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

tokenizer = Tokenizer(inputCol="text_clean", outputCol="words")

In [102]:
dfr = tokenizer.transform(dfr)
#tokenized.select("comments", "words")\
    #.withColumn("tokens", countTokens(col("words"))).show(truncate=False)

In [103]:
dfr.show()

+----+----------+--------+-------------------+-----------+------------------+--------------------+--------------------+--------------+--------------------+
| _c0|listing_id|      id|               date|reviewer_id|     reviewer_name|            comments|          text_clean|average_rating|               words|
+----+----------+--------+-------------------+-----------+------------------+--------------------+--------------------+--------------+--------------------+
|9766|   1271843| 8513212|2013-11-03 00:00:00|    4641577|            Pamela|André was my firs...|andr was my first...|            99|[andr, was, my, f...|
|9767|   1271843| 9043505|2013-12-02 00:00:00|    6890450|              Emma|Andre was the bes...|andre was the bes...|            99|[andre, was, the,...|
|9768|   1271843| 9533868|2014-01-01 00:00:00|    9496791|   Julie And Chris|André was a fanta...|andr was a fantas...|            99|[andr, was, a, fa...|
|9769|   1271843|10049736|2014-01-27 00:00:00|     940929|      

In [104]:
from pyspark.ml.feature import StopWordsRemover

# Define a list of stop words or use default list
remover = StopWordsRemover()
stopwords = remover.getStopWords() 

In [109]:
# Specify input/output columns
remover.setInputCol("words")
remover.setOutputCol("comments_wo_stopwords")

# Transform existing dataframe with the StopWordsRemover
dfr = remover.transform(dfr)

In [110]:
dfr.show()

+----+----------+--------+-------------------+-----------+------------------+--------------------+--------------------+--------------+--------------------+---------------------+
| _c0|listing_id|      id|               date|reviewer_id|     reviewer_name|            comments|          text_clean|average_rating|               words|comments_wo_stopwords|
+----+----------+--------+-------------------+-----------+------------------+--------------------+--------------------+--------------+--------------------+---------------------+
|9766|   1271843| 8513212|2013-11-03 00:00:00|    4641577|            Pamela|André was my firs...|andr was my first...|            99|[andr, was, my, f...| [andr, first, hos...|
|9767|   1271843| 9043505|2013-12-02 00:00:00|    6890450|              Emma|Andre was the bes...|andre was the bes...|            99|[andre, was, the,...| [andre, best, hos...|
|9768|   1271843| 9533868|2014-01-01 00:00:00|    9496791|   Julie And Chris|André was a fanta...|andr was a f

In [16]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol="words", outputCol="features")

model = cv.fit(tokenized)

result = model.transform(tokenized)
result.show(truncate=False)

+----+----------+--------+-------------------+-----------+------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------

In [17]:
result.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- listing_id: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- reviewer_id: integer (nullable = true)
 |-- reviewer_name: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)



#### -----

In [22]:
daa = spark.createDataFrame([([1, 2], {"key": "value"})], ["l", "d"])

In [23]:
daa.show()

+------+--------------+
|     l|             d|
+------+--------------+
|[1, 2]|[key -> value]|
+------+--------------+



In [24]:
daa.printSchema()

root
 |-- l: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- d: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [28]:
daa.select(daa.l.getItem(0)).show()

+----+
|l[0]|
+----+
|   1|
+----+



#### -----

In [ ]:
from pyspark.sql.functions import col,when
orderitems.withColumn("listing_id",
          when(col("order_item_subtotal") != (col("order_item_product_price") * col("order_item_quantity")),"N")
          .otherwise("Y")).show()

In [ ]:
orderitems.withColumn("valid",
          when(col("order_item_subtotal") != (col("order_item_product_price") * col("order_item_quantity")),"N")
          .otherwise("Y")).show()

In [42]:
from pyspark.sql.functions import col,when

In [45]:
result.withColumn("listing_id",when(col("reviewer_id") != (col("features") * col("average_rating")),"N")\
                  .otherwise("Y")).show()

AnalysisException: "cannot resolve '(`features` * CAST(`average_rating` AS DOUBLE))' due to data type mismatch: differing types in '(`features` * CAST(`average_rating` AS DOUBLE))' (struct<type:tinyint,size:int,indices:array<int>,values:array<double>> and double).;;\n'Project [_c0#214, CASE WHEN NOT (reviewer_id#218 = (features#329 * cast(average_rating#232 as double))) THEN N ELSE Y END AS listing_id#444, id#216, date#217, reviewer_id#218, reviewer_name#219, comments#220, average_rating#232, words#313, features#329]\n+- Project [_c0#214, listing_id#215, id#216, date#217, reviewer_id#218, reviewer_name#219, comments#220, average_rating#232, words#313, UDF(words#313) AS features#329]\n   +- Project [_c0#214, listing_id#215, id#216, date#217, reviewer_id#218, reviewer_name#219, comments#220, average_rating#232, UDF(comments#220) AS words#313]\n      +- Project [_c0#214, listing_id#215, id#216, date#217, reviewer_id#218, reviewer_name#219, comments#220, average_rating#232]\n         +- Jo

In [29]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

In [32]:
firstelement=udf(lambda x, y: x*y,FloatType())

In [34]:
def col_mul(col1,col2):
    return col1*col2

In [35]:
mul = udf(col_mul)

In [39]:
res = result.withColumn(mul(f.col("features"),f.col('average_rating')))

TypeError: withColumn() missing 1 required positional argument: 'col'

In [33]:
result.select(firstelement('features','average_rating')).show()

Py4JJavaError: An error occurred while calling o541.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 1 times, most recent failure: Lost task 0.0 in stage 33.0 (TID 627, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 345, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 334, in _batched
    for item in iterator:
 

In [27]:
result.select(result.features[0]).show()

AnalysisException: "Can't extract value from features#329: need struct type but got struct<type:tinyint,size:int,indices:array<int>,values:array<double>>;"

In [18]:
w = result.select(f.col("features"))

In [19]:
w.show()

+--------------------+
|            features|
+--------------------+
|(36166,[0,1,2,3,4...|
|(36166,[0,1,2,3,4...|
|(36166,[0,1,2,3,5...|
|(36166,[0,1,2,3,4...|
|(36166,[0,1,2,3,5...|
|(36166,[0,1,2,6,7...|
|(36166,[0,1,2,3,4...|
|(36166,[0,1,3,5,6...|
|(36166,[0,2,3,5,7...|
|(36166,[4,6,9,31,...|
|(36166,[0,1,3,5,6...|
|(36166,[0,1,2,3,4...|
|(36166,[0,1,2,3,4...|
|(36166,[0,1,3,4,5...|
|(36166,[0,1,3,5,9...|
|(36166,[0,1,2,3,5...|
|(36166,[0,1,2,3,4...|
|(36166,[0,1,2,3,4...|
|(36166,[0,1,2,3,6...|
|(36166,[1,2,3,5,6...|
+--------------------+
only showing top 20 rows



In [49]:
#w = tokenized.select(f.col("words"))

In [57]:
from pyspark.mllib.linalg.distributed import *

In [67]:
from pyspark.mllib.linalg.distributed import *

def as_block_matrix(rdd, rowsPerBlock=1024, colsPerBlock=1024):
    return IndexedRowMatrix(
        rdd.zipWithIndex().map(lambda xi: IndexedRow(xi[1], xi[0]))
    ).toBlockMatrix(rowsPerBlock, colsPerBlock)

#as_block_matrix(rows_1).multiply(as_block_matrix(rows_2))

In [84]:
[w*result.select(f.col("")) list(w.collect()[0][0])

SyntaxError: invalid syntax (<ipython-input-84-c6b4ffdb9a0d>, line 1)

In [73]:
result.withColumn("words",f.col("average_rating")*f.col("features")[0]).show()

AnalysisException: "Can't extract value from features#1542: need struct type but got struct<type:tinyint,size:int,indices:array<int>,values:array<double>>;"

In [ ]:
def extract_title(name):
    name_parts = name.split(',')name.split(',')
    title_and_first = name_parts[1].split('.')
    return title_and_first[0]

extract_title_udf = f.udf(extract_title)

df.withColumn('Title', extract_title_udf(f.col('Name'))).select('Name', 'Title').show()

A simple way to assess how positive each word is:
- for every review: tokenize sentence
- create a bag of words --> a document-term matrix
- create a dictionary, containing every word from the reviews and create an aggregated score on each word. The aggreated word review could be as simple as:
    - adding all the average reviews and dividing with the total number of reviews it has been a part of (to account for non-frequent words) 
    - creating a much simpler score, where bad reviews= -1, middle= 0, high= 1. These simple metrics can then be added up and normalised again. 

In [ ]:
dfl.groupBy('country').agg(f.sum('review_scores_rating')).filter(f.col('country') == 'Sweden').explain()

In [ ]:
dfl.filter(f.col('country') == 'Sweden').groupBy('country').agg(f.sum('review_scores_rating')).explain()